In [1]:
import numpy as np

from Solver import mesh_less, crank_nicolson_fd
from Utilz import F_v0, F_v0_x, diff, eval_F

import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import matplotlib.gridspec as gridspec

In [2]:
def mu(x, t, prms):
    T = prms[0]
    sigma = prms[1]
    a = prms[2]
    v0 = prms[3]
    T_infty = prms[4]
    
    return T*T_infty/(a*(T_infty - T*t)) * (a/(2*T_infty) * (2*x-1) - v0)

In [3]:
def g(x, t, prms, max_k=20):
    T = prms[0]
    sigma = prms[1]
    a = prms[2]
    v0 = prms[3]
    T_infty = prms[4]
    
    sigma_hat = (T*T_infty*sigma**2)/(2*a**2*(T_infty-T))

    if t == 0:
        return 0
    elif np.isclose(mu(x, t, prms) - mu(0, 0, prms), 0):
        return 0
    else:
        return (mu(x, t, prms) - mu(0, 0, prms)) * F_v0_x(x, t, sigma_hat, mu(0, 0, prms), max_k)

# Error Estimation:

In [4]:
sigma = 1
a = 1.5
v0 = 1.1
T_infty = 2

M_ml_1 = 3
M_fd = M_ml_1

time_step = 10

max_k = 20

xx = np.linspace(0, 1, 21)
theta = 0.5

for T in [.2, .4, .6, .8, 1.0]:
    prms = [T, sigma, a, v0, T_infty]
    kappa = (T*T_infty*sigma**2)/(2*a**2*(T_infty-T))
    
    ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.1)
    ml_2 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.4)
    ml_3 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.7)
    
    fd = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=M_fd, N=time_step, max_k=max_k)
    print('Exact solution ...')
    fd_ex = np.load('_exact/exact_LC_{}.npy'.format(int(T*10)), allow_pickle=True)[()]
    
    exact = interp1d(fd_ex['X'], np.concatenate((np.array([0]), fd_ex['sol'], np.array([0]))), 'quadratic')
    fdp = interp1d(fd['X'], np.concatenate((np.array([0]), fd['sol'], np.array([0]))), 'quadratic')
    
    fd_err = np.sqrt(np.sum([np.abs(fdp(x) - exact(x))**2 for x in xx]))
    
    err_ml_1 = np.sqrt(np.sum([np.abs(eval_F(ml_1, x) - exact(x))**2 for x in xx]))
    err_ml_2 = np.sqrt(np.sum([np.abs(eval_F(ml_2, x) - exact(x))**2 for x in xx]))
    err_ml_3 = np.sqrt(np.sum([np.abs(eval_F(ml_3, x) - exact(x))**2 for x in xx]))
    
    print('T = {}, ML_1_ERR = {}, ML_2_ERR = {}, and ML_3_ERR = {}'.format(T, err_ml_1, err_ml_2, err_ml_3))
    print('FD_ERR = {},'.format(fd_err))
    print("\t\t\t\t....................................")

Exact solution ...
T = 0.2, ML_1_ERR = 0.008258133897495858, ML_2_ERR = 0.008016474341239787, and ML_3_ERR = 0.007535124016610333
FD_ERR = 0.015248408859134399,
				....................................
Exact solution ...
T = 0.4, ML_1_ERR = 0.008315922265165154, ML_2_ERR = 0.00805645647352649, and ML_3_ERR = 0.00792965641949162
FD_ERR = 0.015370495020323413,
				....................................
Exact solution ...
T = 0.6, ML_1_ERR = 0.008461338168885257, ML_2_ERR = 0.008533285855143218, and ML_3_ERR = 0.008995112590971818
FD_ERR = 0.01349016099701169,
				....................................
Exact solution ...
T = 0.8, ML_1_ERR = 0.00366741445766276, ML_2_ERR = 0.003904621136780976, and ML_3_ERR = 0.004434357986960888
FD_ERR = 0.00708765743971625,
				....................................
Exact solution ...
T = 1.0, ML_1_ERR = 0.00445391408187274, ML_2_ERR = 0.004271625893846787, and ML_3_ERR = 0.004008786290594353
FD_ERR = 0.003206476115114471,
				.................................

In [465]:
sigma = 1
a = 1.5
v0 = 1.1
T_infty = 2

time_step = 6

max_k = 20

xx = np.linspace(0, 1, 21)
theta = 0.5
T = 0.2

m1 = 5
m2 = 6

prms = [T, sigma, a, v0, T_infty]
kappa = (T*T_infty*sigma**2)/(2*a**2*(T_infty-T))

ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=m1, N=time_step, max_k=max_k, eps=7)
ml_2 = mesh_less(kappa, prms, mu, g, theta=theta, M=m2, N=time_step, max_k=max_k, eps=7)

fd_1 = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=m1, N=time_step, max_k=max_k)
fd_2 = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=m2, N=time_step, max_k=max_k)
print('Exact solution ...')
fd_ex = np.load('_exact/exact_LC_{}.npy'.format(int(T*10)), allow_pickle=True)[()]

exact = interp1d(fd_ex['X'], np.concatenate((np.array([0]), fd_ex['sol'], np.array([0]))), 'quadratic')
fdp_1 = interp1d(fd_1['X'], np.concatenate((np.array([0]), fd_1['sol'], np.array([0]))), 'quadratic')
fdp_2 = interp1d(fd_2['X'], np.concatenate((np.array([0]), fd_2['sol'], np.array([0]))), 'quadratic')

fd_err_1 = np.sqrt(np.sum([(fdp_1(x) - exact(x))**2 for x in xx]))
fd_err_2 = np.sqrt(np.sum([(fdp_2(x) - exact(x))**2 for x in xx]))

err_ml_1 = np.sqrt(np.sum([np.abs(eval_F(ml_1, x) - exact(x))**2 for x in xx]))
err_ml_2 = np.sqrt(np.sum([np.abs(eval_F(ml_2, x) - exact(x))**2 for x in xx]))

Exact solution ...


In [466]:
fd_err_1, fd_err_2, np.log2(fd_err_1/fd_err_2)

(0.004493970174415931, 0.002792565767511808, 0.6863992917563367)

In [467]:
err_ml_1, err_ml_2, np.log2(err_ml_1/err_ml_2)

(0.005387484657598379, 0.0025607655673593223, 1.073036672402914)

# Time Evaluation:

In [4]:
import timeit

In [15]:
sigma = 1
a = 1.5
v0 = 1.1
T_infty = 2

M_ml_1 = 15
M_fd = M_ml_1

time_step = 10

max_k = 20

xx = np.linspace(0, 1, 21)
theta = 0.5       

start = timeit.default_timer()

for _ in range(10):
    for T in np.arange(0, 1+1/15, 1/15)[1:]:
        prms = [T, sigma, a, v0, T_infty]
        kappa = (T*T_infty*sigma**2)/(2*a**2*(T_infty-T))

#         ml_1 = mesh_less(kappa, prms, mu, g, theta=theta, M=M_ml_1, N=time_step, max_k=max_k, eps=0.4)
    
        fd = crank_nicolson_fd(kappa, prms, mu, g, theta=0.5, M=M_fd, N=time_step, max_k=max_k)
        fdp = interp1d(fd['X'], np.concatenate((np.array([0]), fd['sol'], np.array([0]))), 'quadratic')


stop = timeit.default_timer()

print('Time: ', (stop - start)/10)

Time:  5.523177379200206
